# Exploratory Data Analysis 
### @Author : Woojin Park, Nidhi Bhaskar
### @Copyright : 2020, Neolth NSF grant NLP project 
### @Email : woojinpa@andrew.cmu.edu , nidhibha@andrew.cmu.edu
### @Status : In-Progress

In [41]:
### Import Relevant Libraries
import os
import pandas as pd
import numpy as np
import collections
import datetime as dt
import requests
import json
import re
import time

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
from scipy.stats import norm

import string
import re
import nltk
from nltk import pos_tag,word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [40]:
### Build a get_date function to convert date format
#### Build a data_creation function to read json data into pandas dataframe

def get_date(created):
    return dt.datetime.fromtimestamp(created)

def data_creation(subreddit) : 
    with open('submissions_'+subreddit+'.json') as f:
        data = json.loads("[" + 
            f.read().replace("}\n{", "},\n{") + 
        "]")
    data =pd.DataFrame(data)
    reddit_data = data[['author','over_18','title','selftext','num_comments', 'score', 'full_link','created_utc']]
    reddit_data = reddit_data.dropna()
    _timestamp = reddit_data["created_utc"].apply(get_date)
    reddit_data = reddit_data.assign(timestamp = _timestamp)
    reddit_data['over_18'] = reddit_data['over_18'].astype('str')
    reddit_data['subreddit']= subreddit
    # Do one more extra cleaning : keep updating
    #reddit_data=reddit_data[~reddit_data['selftext'].isin([ '[removed]', '[deleted]',''])]
    #subreddit = reddit_data
    
    return subreddit

In [4]:
### Dataframing 4 subreddit Datasets
SuicideWatch_df = data_creation('SuicideWatch')
depressed_df = data_creation('depressed')
happy_df = data_creation('happy')
selfimprovement_df = data_creation('selfimprovement')

In [ ]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [42]:
### First check the datasize and information(type,null,cilumns) of each data frame

In [43]:
SuicideWatch_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47701 entries, 0 to 47794
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   author        47701 non-null  object        
 1   over_18       47701 non-null  object        
 2   title         47701 non-null  object        
 3   selftext      47701 non-null  object        
 4   num_comments  47701 non-null  int64         
 5   score         47701 non-null  int64         
 6   full_link     47701 non-null  object        
 7   created_utc   47701 non-null  int64         
 8   timestamp     47701 non-null  datetime64[ns]
 9   subreddit     47701 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 4.0+ MB


In [44]:
depressed_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26588 entries, 0 to 26639
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   author        26588 non-null  object        
 1   over_18       26588 non-null  object        
 2   title         26588 non-null  object        
 3   selftext      26588 non-null  object        
 4   num_comments  26588 non-null  int64         
 5   score         26588 non-null  int64         
 6   full_link     26588 non-null  object        
 7   created_utc   26588 non-null  int64         
 8   timestamp     26588 non-null  datetime64[ns]
 9   subreddit     26588 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 2.2+ MB


In [45]:
happy_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40941 entries, 0 to 41420
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   author        40941 non-null  object        
 1   over_18       40941 non-null  object        
 2   title         40941 non-null  object        
 3   selftext      40941 non-null  object        
 4   num_comments  40941 non-null  int64         
 5   score         40941 non-null  int64         
 6   full_link     40941 non-null  object        
 7   created_utc   40941 non-null  int64         
 8   timestamp     40941 non-null  datetime64[ns]
 9   subreddit     40941 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 3.4+ MB


In [46]:
selfimprovement_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44196 entries, 0 to 44441
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   author        44196 non-null  object        
 1   over_18       44196 non-null  object        
 2   title         44196 non-null  object        
 3   selftext      44196 non-null  object        
 4   num_comments  44196 non-null  int64         
 5   score         44196 non-null  int64         
 6   full_link     44196 non-null  object        
 7   created_utc   44196 non-null  int64         
 8   timestamp     44196 non-null  datetime64[ns]
 9   subreddit     44196 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 3.7+ MB


In [24]:
### Concat all 4 dataframes into one merged file
all_subreddit_df = pd.concat([SuicideWatch_df,depressed_df,happy_df,selfimprovement_df])
all_subreddit_df.head(2)

author over_18                                              title  \
0    DespressoCafe   False  I don't know where to go or what to do. I can'...   
1  LifeisCrumbling   False                   I'm having an existencial crisis   

                                            selftext  num_comments  score  \
0  Let's make it quick. I'm almost 20. I've been ...             5      1   
1  If I only helped people either as a defense me...             1      1   

                                           full_link  created_utc  \
0  https://www.reddit.com/r/SuicideWatch/comments...   1579669252   
1  https://www.reddit.com/r/SuicideWatch/comments...   1579669256   

            timestamp     subreddit  
0 2020-01-22 00:00:52  SuicideWatch  
1 2020-01-22 00:00:56  SuicideWatch

In [47]:
# Check the info of all dataset
all_subreddit_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159426 entries, 0 to 44441
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   author        159426 non-null  object        
 1   over_18       159426 non-null  object        
 2   title         159426 non-null  object        
 3   selftext      159426 non-null  object        
 4   num_comments  159426 non-null  int64         
 5   score         159426 non-null  int64         
 6   full_link     159426 non-null  object        
 7   created_utc   159426 non-null  int64         
 8   timestamp     159426 non-null  datetime64[ns]
 9   subreddit     159426 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 13.4+ MB


In [26]:
# Check the descriptive statistics
all_subreddit_df.describe(include='all').transpose()

count  unique  \
author        159426   87784   
over_18       159426       2   
title         159426  134509   
selftext      159426   96067   
num_comments  159426     NaN   
score         159426     NaN   
full_link     159426  144975   
created_utc   159426     NaN   
timestamp     159426  144666   
subreddit     159426       4   

                                                            top    freq  \
author                                                [deleted]    6737   
over_18                                                   False  159081   
title                                                      Help     214   
selftext                                                          32518   
num_comments                                                NaN     NaN   
score                                                       NaN     NaN   
full_link     https://www.reddit.com/r/depressed/comments/g2...       3   
created_utc                                                 NaN     NaN   
timestamp                                   2019-12-27 02:35:58       6   
subreddit                                          SuicideWatch   47701   

                           first                last         mean  \
author                       NaT                 NaT          NaN   
over_18                      NaT                 NaT          NaN   
title                        NaT                 NaT          NaN   
selftext                     NaT                 NaT          NaN   
num_comments                 NaT                 NaT      6.32172   
score                        NaT                 NaT      64.4248   
full_link                    NaT                 NaT          NaN   
created_utc                  NaT                 NaT  1.55793e+09   
timestamp    2010-08-28 21:21:56 2020-05-22 14:39:15          NaN   
subreddit                    NaT                 NaT          NaN   

                      std          min          25%          50%         75%  \
author                NaN          NaN          NaN          NaN         NaN   
over_18               NaN          NaN          NaN          NaN         NaN   
title                 NaN          NaN          NaN          NaN         NaN   
selftext              NaN          NaN          NaN          NaN         NaN   
num_comments      31.6846            0            1            2           5   
score             949.752            0            1            1           3   
full_link             NaN          NaN          NaN          NaN         NaN   
created_utc   3.41017e+07  1.28304e+09  1.53827e+09  1.57128e+09  1.5843e+09   
timestamp             NaN          NaN          NaN          NaN         NaN   
subreddit             NaN          NaN          NaN          NaN         NaN   

                      max  
author                NaN  
over_18               NaN  
title                 NaN  
selftext              NaN  
num_comments         2298  
score               82407  
full_link             NaN  
created_utc   1.59017e+09  
timestamp             NaN  
subreddit             NaN

In [53]:
print("SuicideWatch_df Before Cleaning :" , SuicideWatch_df.count(), "\n")
print("SuicideWatch_df After Cleaning :" , SuicideWatch_df[~SuicideWatch_df['selftext'].isin([ '[removed]', '[deleted]',''])].count())

SuicideWatch_df Before Cleaning : author          47701
over_18         47701
title           47701
selftext        47701
num_comments    47701
score           47701
full_link       47701
created_utc     47701
timestamp       47701
subreddit       47701
dtype: int64 

SuicideWatch_df After Cleaning : author          42229
over_18         42229
title           42229
selftext        42229
num_comments    42229
score           42229
full_link       42229
created_utc     42229
timestamp       42229
subreddit       42229
dtype: int64


In [55]:
print("depressed_df Before Cleaning :" , depressed_df.count(), "\n")
print("depressed_df After Cleaning :" , depressed_df[~depressed_df['selftext'].isin([ '[removed]', '[deleted]',''])].count())

depressed_df Before Cleaning : author          26588
over_18         26588
title           26588
selftext        26588
num_comments    26588
score           26588
full_link       26588
created_utc     26588
timestamp       26588
subreddit       26588
dtype: int64 

depressed_df After Cleaning : author          22472
over_18         22472
title           22472
selftext        22472
num_comments    22472
score           22472
full_link       22472
created_utc     22472
timestamp       22472
subreddit       22472
dtype: int64


In [54]:
print("happy_df Before Cleaning :" , happy_df.count(), "\n")
print("happy_df After Cleaning :" , happy_df[~happy_df['selftext'].isin([ '[removed]', '[deleted]',''])].count())

happy_df Before Cleaning : author          40941
over_18         40941
title           40941
selftext        40941
num_comments    40941
score           40941
full_link       40941
created_utc     40941
timestamp       40941
subreddit       40941
dtype: int64 

happy_df After Cleaning : author          7008
over_18         7008
title           7008
selftext        7008
num_comments    7008
score           7008
full_link       7008
created_utc     7008
timestamp       7008
subreddit       7008
dtype: int64


In [ ]:
### We rather scrape more data from happy 
#### or we can find out better subreddit 



















In [56]:
print("selfimprovement_df Before Cleaning :" , selfimprovement_df.count(), "\n")
print("selfimprovement_df After Cleaning :" , selfimprovement_df[~selfimprovement_df['selftext'].isin([ '[removed]', '[deleted]',''])].count())

selfimprovement_df Before Cleaning : author          44196
over_18         44196
title           44196
selftext        44196
num_comments    44196
score           44196
full_link       44196
created_utc     44196
timestamp       44196
subreddit       44196
dtype: int64 

selfimprovement_df After Cleaning : author          37620
over_18         37620
title           37620
selftext        37620
num_comments    37620
score           37620
full_link       37620
created_utc     37620
timestamp       37620
subreddit       37620
dtype: int64
